In [1]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

Because memory constraints limit the batch size in epochsG, I will need to process incrementally.

In [3]:
# base batches.  can be further broken up by splitBatch
eg_batches_50 = [(s,f) for s,f in zip(range(0, 876_978-50_000,50_000),
                      range(50_000, 876978,50_000))] + [(850_000,876979)]
eg_batches_100 = [(s,f) for s,f in zip(range(0, 876_978-100_000,100_000),
                      range(100_000, 876978,100_000))] + [(800_000,876979)]

ratings_histories = []
players_dict = {}

In [4]:
def ceildiv(a, b):
    return -(a // -b)

In [5]:
def splitBatch(atuple, divisions):
    '''Break up batches into smaller pieces to allow epochsG to run.'''
    batch_size = ceildiv((atuple[1] - atuple[0]),divisions)
    if batch_size < (atuple[1] - atuple[0]):
        batches = [b for b in zip(range(atuple[0],atuple[1]-batch_size,batch_size),
             range(atuple[0]+batch_size,atuple[1],batch_size))]
        extra = [(batches[-1][1],atuple[1])]
        return batches + extra
    else:
        return atuple

In [6]:
%%time
for batch in eg_batches_50[0:1]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 10.9 s, sys: 51.7 ms, total: 10.9 s
Wall time: 10.9 s


In [7]:
%%time

for batch in eg_batches_50[1:2]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.17 s, sys: 50.5 ms, total: 4.22 s
Wall time: 4.17 s


In [8]:
%%time

for batch in eg_batches_50[2:3]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.23 s, sys: 23.5 ms, total: 4.25 s
Wall time: 4.24 s


In [9]:
eg_batches_50[3]

(150000, 200000)

In [10]:
b = splitBatch(eg_batches_50[3],2); b0 = b[0]; b1 = b[1]
b

[(150000, 175000), (175000, 200000)]

In [11]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.02 s, sys: 11.8 ms, total: 2.03 s
Wall time: 2.03 s


In [12]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2 s, sys: 11.4 ms, total: 2.01 s
Wall time: 2.01 s


In [13]:
splitBatch(eg_batches_50[4],3)

[(200000, 216667), (216667, 233334), (233334, 250000)]

In [14]:
b = splitBatch(eg_batches_50[4],4); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3]
b

[(200000, 212500), (212500, 225000), (225000, 237500), (237500, 250000)]

In [15]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.1 s, sys: 8.09 ms, total: 1.11 s
Wall time: 1.1 s


In [16]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.02 s, sys: 7.49 ms, total: 1.03 s
Wall time: 1.03 s


In [17]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.08 s, sys: 6.8 ms, total: 1.08 s
Wall time: 1.08 s


In [18]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.15 s, sys: 9.97 ms, total: 1.16 s
Wall time: 1.16 s


In [19]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[5][0]:eg_batches_50[5][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.84 s, sys: 13.9 ms, total: 3.86 s
Wall time: 3.86 s


%%time
players_dict,rh = epochsG(matches[eg_batches_50[6][0]:eg_batches_50[6][1]],players_dict,365)
ratings_histories += [rh]

In [20]:
b = splitBatch(eg_batches_50[6],8); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3];
b4 = b[0]; b5 = b[1]; b6 = b[2]; b7=b[3]
b

[(300000, 306250),
 (306250, 312500),
 (312500, 318750),
 (318750, 325000),
 (325000, 331250),
 (331250, 337500),
 (337500, 343750),
 (343750, 350000)]

In [21]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 835 ms, sys: 12.1 ms, total: 847 ms
Wall time: 839 ms


In [22]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 889 ms, sys: 8.78 ms, total: 897 ms
Wall time: 893 ms


In [23]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 752 ms, sys: 7.04 ms, total: 759 ms
Wall time: 757 ms


In [24]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 843 ms, sys: 7.31 ms, total: 851 ms
Wall time: 848 ms


In [25]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 840 ms, sys: 14.1 ms, total: 854 ms
Wall time: 843 ms


In [26]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 907 ms, sys: 17.1 ms, total: 924 ms
Wall time: 911 ms


In [27]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 794 ms, sys: 12.9 ms, total: 807 ms
Wall time: 798 ms


In [28]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 839 ms, sys: 7.04 ms, total: 846 ms
Wall time: 844 ms


%%time

players_dict,rh = epochsG(matches[eg_batches_50[7][0]:eg_batches_50[7][1]],players_dict,365)
ratings_histories += [rh]

In [29]:
b = splitBatch(eg_batches_50[7],7); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3 = b[3]; b4 = b[4]; b5 = b[5];
b6 = b[6]


In [30]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 909 ms, sys: 7.35 ms, total: 916 ms
Wall time: 913 ms


In [31]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 860 ms, sys: 7 ms, total: 867 ms
Wall time: 864 ms


In [32]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 956 ms, sys: 13.1 ms, total: 969 ms
Wall time: 960 ms


In [33]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 952 ms, sys: 8.67 ms, total: 961 ms
Wall time: 957 ms


In [34]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 989 ms, sys: 10.5 ms, total: 999 ms
Wall time: 994 ms


In [35]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 867 ms, sys: 8.91 ms, total: 876 ms
Wall time: 871 ms


In [36]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 983 ms, sys: 10.9 ms, total: 994 ms
Wall time: 988 ms


In [37]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.01 s, sys: 17.3 ms, total: 1.02 s
Wall time: 1.01 s


In [38]:
b = splitBatch(eg_batches_50[8],2); b0 = b[0]; b1 = b[1]
b

[(400000, 425000), (425000, 450000)]

In [39]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.14 s, sys: 23.8 ms, total: 2.16 s
Wall time: 2.15 s


In [40]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.94 s, sys: 10 ms, total: 1.95 s
Wall time: 1.94 s


In [41]:
b = splitBatch(eg_batches_50[9],8); b0 = b[0]; b1 = b[1]; 
b2 = b[2];b3 = b[3];b4 =b[4]; b5 = b[5]; b6 = b[6]; b7 = b[7]
b

[(450000, 456250),
 (456250, 462500),
 (462500, 468750),
 (468750, 475000),
 (475000, 481250),
 (481250, 487500),
 (487500, 493750),
 (493750, 500000)]

In [42]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 868 ms, sys: 8.86 ms, total: 877 ms
Wall time: 873 ms


In [43]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 971 ms, sys: 10.7 ms, total: 982 ms
Wall time: 976 ms


In [44]:
b = splitBatch(b2,2)
b2_0 = b[0]; b2_1 = b[1];

In [45]:
%%time
players_dict,rh = epochsG(matches[b2_0[0]:b2_0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 757 ms, sys: 8.43 ms, total: 765 ms
Wall time: 761 ms


In [46]:
%%time
players_dict,rh = epochsG(matches[b2_1[0]:b2_1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 801 ms, sys: 23.7 ms, total: 824 ms
Wall time: 803 ms


In [47]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 921 ms, sys: 9.3 ms, total: 931 ms
Wall time: 927 ms


In [48]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 832 ms, sys: 6.93 ms, total: 839 ms
Wall time: 836 ms


In [49]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.01 s, sys: 7.42 ms, total: 1.02 s
Wall time: 1.02 s


In [50]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.03 s, sys: 6.3 ms, total: 1.03 s
Wall time: 1.03 s


In [51]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 990 ms, sys: 9.37 ms, total: 1e+03 ms
Wall time: 995 ms


In [52]:
b = splitBatch(eg_batches_50[10],2); b0 = b[0]; b1 = b[1]
b

[(500000, 525000), (525000, 550000)]

In [53]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.81 s, sys: 8.4 ms, total: 1.82 s
Wall time: 1.82 s


In [54]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.9 s, sys: 7.39 ms, total: 1.91 s
Wall time: 1.91 s


In [55]:
b = splitBatch(eg_batches_50[11],5); b0 = b[0]; b1 = b[1];
b2 = b[2]; b3 = b[3];b4 = b[4];
b

[(550000, 560000),
 (560000, 570000),
 (570000, 580000),
 (580000, 590000),
 (590000, 600000)]

In [56]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.15 s, sys: 7.85 ms, total: 1.16 s
Wall time: 1.16 s


In [57]:
b = splitBatch(b2,2)
b1_0 = b[0]; b1_1 = b[1];

In [58]:
%%time
players_dict,rh = epochsG(matches[b1_0[0]:b1_0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 932 ms, sys: 10.5 ms, total: 943 ms
Wall time: 936 ms


In [59]:
%%time
players_dict,rh = epochsG(matches[b1_1[0]:b1_1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 801 ms, sys: 13.9 ms, total: 815 ms
Wall time: 805 ms


In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [61]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.29 s, sys: 9.66 ms, total: 1.3 s
Wall time: 1.29 s


In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[1]:b[-1][1]],players_dict,365)
ratings_histories += [rh]

In [65]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[13][0]:eg_batches_50[13][1]],players_dict,365)
ratings_histories += [rh]

KeyboardInterrupt: 

In [66]:
b = splitBatch(eg_batches_50[14],2); b0 = b[0]; b1 = b[1]
b

[(700000, 725000), (725000, 750000)]

In [67]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

KeyboardInterrupt: 

In [68]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

KeyboardInterrupt: 

In [69]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[15][0]:eg_batches_50[15][1]],players_dict,365)
ratings_histories += [rh]

ZeroDivisionError: float division by zero

In [70]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[16][0]:eg_batches_50[16][1]],players_dict,365)
ratings_histories += [rh]

ZeroDivisionError: float division by zero

In [71]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[17][0]:eg_batches_50[17][1]],players_dict,365)
ratings_histories += [rh]

KeyboardInterrupt: 